In [1]:
import pandas as pd

df = pd.read_csv('AraPoems_Dataset.csv')
df.fillna('', inplace=True)
display(len(df))



df['group_count'] = df.groupby('link').cumcount()

df['newLink'] = df.apply(lambda x: f"{x['link']}_{x['group_count'] // 5:02d}", axis=1)


def process_group(group):
    count = group['first_hemistich'].count()
    full_sentences = (group['first_hemistich'] + ' # ' + group['second_hemistich']).tolist()
    first_line = full_sentences[0] if len(full_sentences) > 0 else ''
    compiled_text = '\n'.join(full_sentences[1:]) if len(full_sentences) > 1 else ''
    return pd.Series({
        'count': count,
        'input': first_line,
        'outputPoem': compiled_text
    })
    

dfc = df.groupby('newLink').apply(process_group).reset_index()

# dfc = df.groupby('link')['first_hemistich'].count().reset_index(name='count')
dfc.fillna('', inplace=True)
display(len(dfc))
display(dfc[:5])
# dfc = dfc[dfc['link'] != '']
# display(len(dfc))
dfc = dfc[dfc['count'] > 1]
display(len(dfc))


2090907

492624

,newLink,count,input,outputPoem
0,https://poetry.dctabudhabi.ae/#/poems/1-%D8%AE...,5,خليلي لا تستعجلا أن تزودا # وأن تجمعا شملي وتن...,خليلي لا تستعجلا أن تزودا # وأن تجمعا شملي وتن...
1,https://poetry.dctabudhabi.ae/#/poems/1-%D8%AE...,5,لعمرك ما نفس بجد رشيدة # تؤامرني سرا لأصرم مرثدا,وإن ظهرت منه قوارص جمة # وأفرع في لومي مرارا و...
2,https://poetry.dctabudhabi.ae/#/poems/1-%D8%AE...,3,وإن صرحت كحل وهبت عرية # من الريح لم تترك لذي ...,صبرت على وطء الموالي وحطمهم # إذا ضن ذو القربى...
3,https://poetry.dctabudhabi.ae/#/poems/10-%D9%8...,5,هل لا يهيج شوقك الطلل # أم لا يفرط شيخك الغزل,أم ذا القطين أصاب مقتله # منه وخانوه إذا احتمل...
4,https://poetry.dctabudhabi.ae/#/poems/10-%D9%8...,5,تامت فؤادك يوم بينهم # عند التفرق ظبية عطل,شنفت إلى رشأ ترببه # ولها بذات الحاذ معتزل\nظل...


463638

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer



def formatting_prompts_func(examples):
    # instructions = examples["instruction"]
    # instructions = f'generate {} verses in Arabic using the following verse as a starting point'
    counts = examples["count"]
    inputs = examples["input"]
    outputs = examples["outputPoem"]
    texts = []
    # for instruction, input, output in zip(instructions, inputs, outputs):
    for count, input, output in zip(counts, inputs, outputs):
        instructions = f'generate {count} verses in Arabic using the following verse as a starting point'
        text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"
        texts.append(text)
    return {"text": texts}

dataset = Dataset.from_pandas(dfc)
dataset = dataset.map(formatting_prompts_func, batched=True)



dataset = dataset.train_test_split(test_size=0.01)

# display(dataset)

train_dataset = dataset['train']
test_dataset = dataset['test']


model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained('checkpoint/')
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)



i = 0

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

2024-07-11 06:03:13.663835: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 06:03:13.683424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 06:03:13.964504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Map:   0%|          | 0/463638 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: أيام ريب الدهر عني # ساكت والعيش ناطق
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: أيام ريب الدهر عني # ساكت والعيش ناطق
Output:  # 
لقد تمنيتك يا منى # لشبيبتي وأصابتي
فإذا غدوت فلا تكن # مني لقلبي حياتي
إن لم تجد لي منيتي # فأنا مني منيتي
فإذا صرت منيتي # فليس لي منيتي  #  #  #  #  #  #  #  # 

the original output
يا سادة نكثوا العهود # على النوى ونسوا المواثق
الحزن معتقل لدي # ببعدكم والصبر أبق
أنا من بلوتم فعله # وخلائقي تلك الخلائق
فكما علمتم في معا # تبة الأحبة لا أضايق


In [3]:
i = 1

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: فلا تعجلن وانظر بأرضك فارسا # يهز ردينيا وأبيض صارما
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: فلا تعجلن وانظر بأرضك فارسا # يهز ردينيا وأبيض صارما
Output:  وعد عن الحداثة إنها أعمى # فما في الحقيقة من رونق زاهي
وأرأف بالشباب واعلم أن هذا # شؤون الحياة من العيش الرقاق
وأنك فان عاجلا أو قريبا # وأعلم أن حياتك ذات نهاق
و

the original output
له كل يوم غارة عرفت له # إذا قادها للموت جردا سواهما
وعبد بني برشا تركنا مجدلا # غداة ثوى بين الفوارس كازما
تناولته فاختل سيفي ذبابه # شراسيفه العليا وجد المعاصما
وأنت قريب قد رأيت مكانه # تنادي شتيرا يوم ذاك وعاصما


In [4]:
i = 2

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: صديق يداري الحزن عنك مماذق # ودمع يغب العين فيك منافق
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: صديق يداري الحزن عنك مماذق # ودمع يغب العين فيك منافق
Output:  وإن كنت قد أصبحت في حيرة # من الحب لا تأخذك الأشواق
فلا تجزعن منك الحوادث إنما # الحوادث في حياة الفتى أوهام
وإن كنت قد عرفت منك الحب فانظر # إلى العين منك كيف هي واه
وأدرك بفهم

the original output
وقلب إذا عانى الأسى طلب الأسى # لراحته من رق ودك أبق
بكى القاطنون الظاعنون وقوض ال # حلول وصاحت بالفراق النواعق
ولكنني بالأمس لم تسر ناقة # بمختلس مني ولم يحد سائق
سلا عنه في أي المفاوز فاتني # وطرفي له راع وطرفي سابق


In [5]:
i = 3

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: وثارت عليه الروح تطلب ثارها # كما هز قرم في الجلاد مهندا
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: وثارت عليه الروح تطلب ثارها # كما هز قرم في الجلاد مهندا
Output:  ولو أنني كنت المريد لم يكن # لتعرفه إلا المريد المريد
وإن كنت في عين الوجود منية # فإنك لا عين ولا وجودا
فلا تطلبن مني سواه فإنه # إذا قيل من أين الرجاء فما أرى
وأنت بلا ذنب أتيتك مقلدا

the original output
فبات بحمى صدع الرأس جمرها # كما أبرق الغيم الجهام وأرعدا
ووافى أباها غاديا وهو حامل # حديدا طليق الشفرتين محددا
وقال له يا مذبلا زهر الصبا # ويا جاعلا ماء الشبيبة أركدا
أعد لي سلمى أين سلمى إلم تكن # تفدى بأرواح لكنت لها الفدى


In [6]:
i = 4

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: لولا مغالبة الشجون لخاطري # لنظمت فيك يتيمة الأزمان
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: لولا مغالبة الشجون لخاطري # لنظمت فيك يتيمة الأزمان
Output:  فكأنها في كل يوم منشأة # مني بغيرها على الإحسان
فإذا هممت بمدحك المولى الذي # لم يرضه من أمد الدنيا مكان
فكأنه لما علا فوق السما # في المجد أرسى من قواعد ركنان
وإذا أتيتك في البلاغ

the original output
وأنا الذي أرثي الشموس إذا هوت # فتعود سيرتها إلى الدوران
قد كنت تهتف في الورى بقصائدي # وتجل فوق النيرات مكاني
ماذا دهاني يوم بنت فعقني # فيك القريض وخانني إمكاني
هون عليك فلا شمات بميت # إن المنية غاية الإنسان


In [7]:
i = 5

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: وأطلق رمحه فمضى # وهكطور انحنى حذرا
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: وأطلق رمحه فمضى # وهكطور انحنى حذرا
Output:  وغادر الموت في ثياب # ن الموت فوق السهام
ثم قال هكطور يرنو # ألا يا قوما احذروا
وأقسمت بالهوى أن # لا أستطيع له قساما
وأيقن أن يديه # يأتيها أينما حيا
فيا قوم لا ترهبوا

the original output
فجاوز رأسه للأر # ض لا ينتابه ضرر
ولكن بادرت فالا # س تنزعه على عجل
وترجعه لأخي # ل وعن هكطور تستتر
فصاح فتى الطراود # قد شططت وتدعي زورا


In [8]:
i = 6

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 3 verses in Arabic using the following verse as a starting point
Input: شب الهوى شبة نيران # والفجر قد فجر أجفاني
Output: 

Instruction: generate 3 verses in Arabic using the following verse as a starting point
Input: شب الهوى شبة نيران # والفجر قد فجر أجفاني
Output:  # 
عيناك في عين الغرام #  # 

the original output
ما فعل الشمر على كفره # ما فعلت أجفان شمران
لو لم يكن ثناه في حسنه # أخوه ما كان له ثان


In [9]:
i = 7

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: فغلق أبواب عنوة # وشيد أركان الخنا والمأثم
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: فغلق أبواب عنوة # وشيد أركان الخنا والمأثم
Output:  فما زال يغشى القلوب بالظلم # 
ويمحو عن الحقائق كل نور # 
فلا يبعد اللهه مني سجنه # 
ولا يبعد اللهه مني سجنه #  ويشهدني الحق في الحشر والقسم  5 5 5 5 5 5 5 5 5 5 5

the original output
وأضحى على حب المفاسد عاكفا # فتبا لهذا من
لقد عامل بالجور والجفا # ورش بالشقا والجرائم
ولم يتبع في سنة # على جهله أحيا رسوم المعالم
وأنسى بما أبدى شجاعة عنتر # وإقدام عمرو مع سماحة حاتم


In [10]:
i = 8

instructions = f'generate {test_dataset[i]["count"]} verses in Arabic using the following verse as a starting point'
input = test_dataset[i]["input"]
output = '' 
text = f"Instruction: {instructions}\nInput: {input}\nOutput: {output}"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(text)  #, temperature=0.7
print(text)
print()
print(result[0]['generated_text'])
print()
print(f'the original output\n{test_dataset[i]["outputPoem"]}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: مسحت قذى الدنيا يداك وإنما # كانت على الأيام مسحة أرمد
Output: 

Instruction: generate 5 verses in Arabic using the following verse as a starting point
Input: مسحت قذى الدنيا يداك وإنما # كانت على الأيام مسحة أرمد
Output:  أين الألى قد كان منهم من يرى # فينا ويسمع من نداه ويجتدي
أين الذين إذا التمست ركائب # كنا نرجي أن يصادفوك من اليد
أين الذين إذا أتت خطبة # ألقى الكلام إليك غير مشدد
أين الذين إذا جئنا بالشك

the original output
نالت بنو عبد السلام بك المدى # من كل سابقة وعز سرمد
ضربوا بسيفك هام كل ملمة # سيف عن المعروف ليس بمغمد
ورموا بسهمك عن قسي إصابة # غرض الكمال فكان أي مسدد
القائدين الخيل تعثر بالطلى # عثر الرياح بكل طود أقود
